In [1]:
import sys
sys.path.append('/Users/jpnousu/pyAPES_main')
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from pyAPES.utils.iotools import read_forcing
from pyAPES.soil.soil import Soil_1D
from pyAPES.snow.snow import SnowModel
from pyAPES.snow.srfebal import EnergyBalance
from pyAPES.parameters.mlm_parameters import gpara, cpara, spara, snow # model configuration, canopy parameters, soil parameters
from pyAPES.utils.constants import WATER_DENSITY

KELVIN = 273.15


In [2]:
# Creating Soil_1D
soil = Soil_1D(spara)

# Creating EnergyBalance
eb = EnergyBalance(snow)

# Creating SnowModel
snowpack = SnowModel(snow, soil_dz=soil.grid['dz'][0])

# Manipulating the params to correspond to those of Degerö
gpara['forc_filename'] = r'/Users/jpnousu/pyAPES_main/forcing/Degero/Degero_forcing_2014-2016.dat'
gpara['start_time'] = '2014-09-01'
gpara['end_time'] = '2015-06-01'

In [3]:
# Reading the forcing file
forcing = read_forcing(
    forcing_file=gpara['forc_filename'],
    start_time=gpara['start_time'],
    end_time=gpara['end_time'],
    dt=gpara['dt']
)

forcing['Prec_liquid'] = forcing['Prec'].where(forcing['Tair'] > 1, 0)
forcing['Prec_ice'] = forcing['Prec'].where(forcing['Tair'] <= 1, 0)

In [4]:
# Looping over the forcing file, running forestfloor and soil models for each timestep and saving the results
snow_res = pd.DataFrame()
eb_res = pd.DataFrame()
soil_res = pd.DataFrame()

for index, row in forcing.iterrows():
    

    eb_forcing = {
        'cveg': 0.0,
        'Ds1': 0.05,
        'fcans': 0.0,
        'fsnow': snowpack.fsnow,
        'gs1': 0.013274175113270106,
        'ks1': 0.15868952201804923,
        'lveg': 0.0,
        'LW': row['LWin'],
        'Ps': row['P'], 
        'RH': 100,
        'SWsrf': row['dirNir'] + row['dirPar'],
        'Sveg': 0.0,
        'SWveg': 0.0,
        'Ta': row['Tair']+KELVIN,
        'tdif': 0.0,
        'Ts1': 0.0,
        'Tveg0': 0.0,
        'Ua': row['U'],
        'VAI': 0.0,
        'vegh': 0.0,
        'zT': 0.0,
        'zU': 0.0,
        'Sice': snowpack.Sice,
    }

    eb_fluxes, eb_states = eb.run(dt=gpara['dt'],
                                  forcing=eb_forcing)

    snow_forcing = {  # Reset the dictionary for each iteration
        'drip': 0, # 
        'Esrf': eb_fluxes['evaporation_surface'], # 
        'Gsrf': eb_fluxes['ground_heat'], # m/s
        'ksoil': soil.heat.thermal_conductivity[0],  #
        'Melt': eb_fluxes['melt'],  #
        'Rf': row['Prec_liquid'],  #
        'Sf': row['Prec_ice'],  #
        'Ta': row['Tair']+KELVIN,  #
        'trans': 0,  #
        'Tsrf': eb_states['surface_temperature'],  # 
        'unload': 0, #
        'Tsoil': soil.heat.T[0]+KELVIN, #
    }

    # --- solve snow
    snow_fluxes, snow_states = snowpack.run(
                dt=gpara['dt'],
                forcing=snow_forcing
                )

    ###### Inputs for soil (based on how they are used in pyAPES_MLM.py)
    soil_forcing = {
        'potential_infiltration': snow_fluxes['soil_runoff'] / WATER_DENSITY,
        'potential_evaporation': 0,
        'pond_recharge': 0,
        'atmospheric_pressure_head': -1.0E6, # [m]
        'ground_heat_flux': -snow_fluxes['soil_heat_flux'],
        'date': index
    }
    
    soil_fluxes, soil_states = soil.run(
                dt=gpara['dt'],
                forcing=soil_forcing,
                )
    
    # --- update soil
    soil_states.update(soil_fluxes)

    # Combine results into a single dictionary with all values as floats
    combined_results = {
        'time': index,  # Use the time index from the forcing DataFrame
        **{f'es_{key}': value for key, value in eb_states.items()},
        **{f'ef_{key}': value for key, value in eb_fluxes.items()},  
        **{f'ss_{key}': value for key, value in snow_states.items()},  
        **{f'sf_{key}': value for key, value in snow_fluxes.items()},  
        **{f's_vwc': soil_states['volumetric_water_content'][0]},
        **{f's_temp': soil_states['temperature'][0]},
    }

    # Append the combined results to the results DataFrame
    snow_res = pd.concat([snow_res, pd.DataFrame([combined_results])], ignore_index=True)

snow_res.set_index('time', inplace=True)

Truth
Sice here [0.00960621 0.         0.        ]
Melt here 5.336784714434921e-06
Truth
Sice here [0.00960621 0.         0.        ]
Melt here 5.336784714434921e-06
Truth
Sice here [0.00960621 0.         0.        ]
Melt here 5.336784714434921e-06
Truth
Sice here [0.00960621 0.         0.        ]
Melt here 5.336784714434921e-06
Truth
Sice here [0.00960621 0.         0.        ]
Melt here 5.336784714434921e-06
Truth
Sice here [0.00960621 0.         0.        ]
Melt here 5.336784714434921e-06
Truth
Sice here [0.00960621 0.         0.        ]
Melt here 5.336784714434921e-06
Truth
Sice here [0.00960621 0.         0.        ]
Melt here 5.336784714434921e-06
Truth
Sice here [0.00960621 0.         0.        ]
Melt here 5.336784714434921e-06
Truth
Sice here [0.00960621 0.         0.        ]
Melt here 5.336784714434921e-06
Truth
Sice here [0.00960621 0.         0.        ]
Melt here 5.336784714434921e-06
Truth
Sice here [0.00960621 0.         0.        ]
Melt here 5.336784714434921e-06
Trut

/Users/jpnousu/pyAPES_main/pyAPES/utils/utilities.py:70: RuntimeWarning: invalid value encountered in scalar divide
  U[i] = (D[i] - a[i] * U[i - 1]) / V[i]
/Users/jpnousu/pyAPES_main/pyAPES/utils/utilities.py:71: RuntimeWarning: invalid value encountered in scalar divide
  G[i] = C[i] / V[i]


Truth
Sice here [10.00000093 20.00000186 40.00000372]
Melt here 0.038888892503281675
Truth
Sice here [10.00000093 20.00000186 40.00000372]
Melt here 0.038888892503281675
Truth
Sice here [10.00000093 20.00000186 40.00000372]
Melt here 0.038888892503281675
Truth
Sice here [10.00000093 20.00000186 40.00000372]
Melt here 0.038888892503281675
Truth
Sice here [10.00000093 20.00000186 40.00000372]
Melt here 0.038888892503281675
Truth
Sice here [10.00000093 20.00000186 40.00000372]
Melt here 0.038888892503281675
Truth
Sice here [10.00000093 20.00000186 40.00000372]
Melt here 0.038888892503281675
Truth
Sice here [7.10542736e-15            nan            nan]
Melt here nan


/Users/jpnousu/pyAPES_main/pyAPES/snow/snow.py:414: RuntimeWarning: invalid value encountered in scalar divide
  wt = dnew / D[kold]


UnboundLocalError: cannot access local variable 'Melt' where it is not associated with a value

In [5]:
index

Timestamp('2014-10-14 07:30:00')